In [10]:
# By the way, this is Julia not Python lol

# code adapted from Wiedman et al 2021
# https://github.com/devmotion/Calibration_ICLR2021

using Base.Filesystem

using Arrow
using CairoMakie
using CalibrationErrors
using CalibrationErrorsDistributions
using CalibrationTests
using CSV
using DataFrames
using Distributions
using Flux
using Optim
using Random
using Statistics
using ProgressLogging
using Query

using Logging: with_logger
using TerminalLoggers: TerminalLogger
using ColorSchemes: Dark2_8

pwd()

"/Users/fm03-mb03/Repositories/proper_calibration_errors"

In [11]:
# data downloaded from http://archive.ics.uci.edu/ml/datasets/Residential+Building+Data+Set
data = DataFrame(CSV.read("data/Residential-Building-Data-Set.csv", DataFrame; delim=';', decimal=',', header=true))
data = select(data, Not("V-9"))

data_id = "ResBuild"

function random_split(df, n)
    n_0 = size(df)[1]
    indices = randperm(n_0)
    train_i = indices[1:n]
    eval_i = indices[(n+1):n_0]
    return (df[train_i, :], df[eval_i, :])
end

function split_input_target(df, target, eps=0)
    # potentially add gaussian noise via 'eps'
    ys = select(df, target)
    xs = select(df, Not(target))
    dist = Normal(0, 1)
    n = size(df)[1]
    d = size(df)[2] - 1
    n_samples = n * d
    noise = rand(dist, n_samples) * eps
    pert_xs = Matrix(xs) + reshape(noise, n, d)
    
    return (transpose(pert_xs), vec(Matrix(ys)))
end

split_input_target (generic function with 2 methods)

In [12]:
Random.seed!(100)

n_models = 5

df_train, df_eval = random_split(data, 100);
df_val, df_test = random_split(df_eval, 100);

target = "V-10"
train_data = split_input_target(df_train, target);
val_data = split_input_target(df_val, target);
test_data = split_input_target(df_test, target);

In [13]:
function nn_model()
    ## initial parameters
    f = Chain(
        Dense(107 => 200, relu; init=Flux.glorot_uniform),
        Dense(200 => 2; init=Flux.glorot_uniform),
    )
    # due to me being stupid in julia, the variance output is in the log space
    # and will be transformed in each loss later
    return f
end

# we called this Dawid-Sebastiani-Score (DSS) in the paper
function pmcc(ps, ys)
    vars = abs.(ps[2,:])
    scores = (ps[1,:] .- ys).^2 ./ vars .+ log.(vars)
    return mean(scores)
end

function skce_biased(ps, ys)
    kernel = WassersteinExponentialKernel() ⊗ SqExponentialKernel()
    estimator = BiasedSKCE(kernel)
    n = size(ps, 2)
    predictions = [Normal(ps[1, i], sqrt(abs(ps[2, i]))) for i in 1:n]
    return calibrationerror(estimator, vec(predictions), ys)
end

function skce_unbiased(ps, ys)
    kernel = WassersteinExponentialKernel() ⊗ SqExponentialKernel()
    estimator = UnbiasedSKCE(kernel)
    n = size(ps, 2)
    predictions = [Normal(ps[1, i], sqrt(abs(ps[2, i]))) for i in 1:n]
    return calibrationerror(estimator, vec(predictions), ys)
end

function mse(ps, ys)
    scores = (ps[1,:] .- ys).^2
    return mean(scores)
end

function mean_var(ps)
    vars = abs.(ps[2,:])
    return mean(vars)
end

function var_se_dist(ps, ys)
    # how much does the predicted var with the squared error of the mean prediction correspond?
    # > 1, too pessimistic
    # < 1, too optimistic
    # cheers to reviewer #3 for asking!
    vars = abs.(ps[2,:])
    squared_errors = (ps[1,:] .- ys).^2
    ratios = squared_errors ./ vars
    return mean(ratios), var(ratios)
end

var_se_dist (generic function with 1 method)

In [14]:
epsilon = 1e-10

function recal(f, val_xs, val_ys)
    preds = f(val_xs)
    
    function helper(w)
        cals = transpose(hcat(preds[1, :], abs.(preds[2, :]) .* w[1] .+ w[2]))
        return pmcc(cals, val_ys)
    end

    lower = [epsilon, epsilon]
    upper = [Inf, Inf]
    res = optimize(helper, lower, upper, [1.0, 2*epsilon], Fminbox(LBFGS()))
    w = Optim.minimizer(res)

    return x -> transpose(hcat(x[1, :], abs.(x[2, :]) .* w[1] .+ w[2]))
end

recal (generic function with 1 method)

In [15]:
# helper function for distribution shift
function dist_shift_helper(file, test_xs, ys, models)
    epsilons = (0:5) .* 2000000

    @progress name = "iterations" for (i, eps) in enumerate(epsilons)

        ## compute gradients
        gradients = gradient(Flux.params(test_xs)) do
            return mean_var(models[1](test_xs))
        end
        # FGSM (without sign; makes more sense for tabular data)
        pert_xs = test_xs + gradients[test_xs] * eps

        # FGSM (with sign)
        #pert_xs = test_xs + sign.(gradients[test_xs]) * eps

        preds = models[1](pert_xs)

        ## mean squared error
        mse_v = mse(preds, ys)
        println(file, eps, ",MSE,", mse_v, ",uncal")            

        ## mean predicted var
        var = mean_var(preds)
        println(file, eps, ",Avg Var,", var, ",uncal")

        ## ratio of predicted var and squared error of mean
        avg_ratio, var_ratio = var_se_dist(preds, ys)
        println(file, eps, ",Var SE Dist,", avg_ratio, ",uncal")
        println(file, eps, ",Var SE Dist Var,", var_ratio, ",uncal")

        preds_cal = models[2](models[1](pert_xs))

        ## mean squared error
        mse_v = mse(preds_cal, ys)
        println(file, eps, ",MSE,", mse_v, ",cal")            

        ## mean predicted var
        var = mean_var(preds_cal)
        println(file, eps, ",Avg Var,", var, ",cal")

        ## ratio of predicted var and squared error of mean
        avg_ratio, var_ratio = var_se_dist(preds_cal, ys)
        println(file, eps, ",Var SE Dist,", avg_ratio, ",cal")
        println(file, eps, ",Var SE Dist Var,", var_ratio, ",cal")
    end
end

function evaluate_helper(file, i, preds, ys)

    ## mean squared error
    mse_v = mse(preds, ys)
    println(file, i - 1, ",MSE,", mse_v)            

    ## mean-variance score
    pmcc_v = pmcc(preds, ys)
    println(file, i - 1, ",PMCC,", pmcc_v)

    ## unbiased estimator of SKCE
    skce = skce_unbiased(preds, ys)
    println(file, i - 1, ",SKCE (unbiased),", skce)

    ## biased estimator of SKCE
    skce = skce_biased(preds, ys)
    println(file, i - 1, ",SKCE (biased),", skce)

    ## mean predicted var
    var = mean_var(preds)
    println(file, i - 1, ",Avg Var,", var)

    ## ratio of predicted var and squared error of mean
    avg_ratio, var_ratio = var_se_dist(preds, ys)
    println(file, i - 1, ",Var SE Dist,", avg_ratio)
    println(file, i - 1, ",Var SE Dist Var,", var_ratio)
end

# ## Training
#
# We use a maximum likelihood approach and train the parameters $\theta$ of the model
# for 4000 iterations by minimizing the DSS on the training data set
# using ADAM.
#
# We train 5 models and compute the predicted distributions on the training and test data sets
# in each iteration step.
#
# The initial values of the weight matrices of the neural networks are sampled from the
# [uniform Glorot initialization](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)
# and the offset vectors are initialized with zeros. The model parameters are learnt by
# iteratively minimizing the DSS on the training data set.
# The parameters of the neural networks are trained by gradient descent with the
# [Adam optimization algorithm](https://arxiv.org/pdf/1412.6980.pdf) (default
# settings in [Flux.jl](https://github.com/FluxML/Flux.jl)).


function train(file_shift, id, (train_xs, train_ys), (val_xs, val_ys), (test_xs, test_ys), niters = 2100)
    
    out_train = joinpath("data", data_id, "statistics_id=$(id)_dataset=train.csv")
    out_test = joinpath("data", data_id, "statistics_id=$(id)_dataset=test.csv")
    out_test_rc = joinpath("data", data_id, "statistics_id=$(id)_dataset=test_rc.csv")
    
    for file in [out_train, out_test, out_test_rc]
        mkpath(dirname(file))
    end

    open(out_train, "w") do file_train; open(out_test, "w") do file_test; open(out_test_rc, "w") do file_test_rc

        println(file_train, "iteration,statistic,estimate")
        println(file_test, "iteration,statistic,estimate")
        println(file_test_rc, "iteration,statistic,estimate")

        ## compute the predictions of the initial neural network
        f = nn_model()

        ## train with ADAM
        params = Flux.Params(Flux.params(f))
        opt = ADAM()
        @progress name = "training (id = $id)" for i in 1:(niters + 1)
            ## compute gradients
            gradients = gradient(params) do
                return pmcc(f(train_xs), train_ys)
            end

            ## update the parameters
            Flux.Optimise.update!(opt, params, gradients)

            g = recal(f, val_xs, val_ys)
            evaluate_helper(file_train, i, f(train_xs), train_ys)
            evaluate_helper(file_test, i, f(test_xs), test_ys)
            evaluate_helper(file_test_rc, i, g(f(test_xs)), test_ys)

#             if i == 2001
#                 models = (f, g)
#                 dist_shift_helper(file_shift, test_xs, test_ys, models)
#             end
        end
    end; end; end    
end

train (generic function with 2 methods)

In [16]:
out_shift = "data/dist_shift_uci_fgm.csv"
mkpath(dirname(out_shift))

open(out_shift, "w") do file_shift
    println(file_shift, "epsilon,statistic,estimate,model")

    Random.seed!(100)
    for (id, seed) in enumerate(rand(UInt, n_models))
        @info "training NN model: run $id"
        Random.seed!(seed)
        train(file_shift, id, train_data, val_data, test_data)
    end
end

┌ Info: training NN model: run 1
└ @ Main In[16]:9
┌ Info: training NN model: run 2
└ @ Main In[16]:9
┌ Info: training NN model: run 3
└ @ Main In[16]:9
┌ Info: training NN model: run 4
└ @ Main In[16]:9
┌ Info: training NN model: run 5
└ @ Main In[16]:9


In [47]:
# SOME TESTS FOR Dawid-Sebastiani Score

In [240]:
mu = 0
var = 1e7

d = Normal(mu, var^.5)
n = 1


In [244]:
function _func(n)
    y = rand(d, n);
    x = transpose(hcat(zero(y) .+ mu, zero(y) .+ var))
    best = pmcc(x, y)
    #println(best)

    x = transpose(hcat(zero(y) .+ mu, zero(y) .+ var ./ 0.8))
    other = pmcc(x, y)
    #println(other)

    return best < other
end    

_func (generic function with 2 methods)

In [278]:
mean([_func(1000) for _ in 1:10000])

0.9928